# Disease enrichment analysis of phenotypic avatars

In [1]:
from pyhpo import Ontology, HPOSet, Omim
from pyhpo import stats as hpostats
from typing import List, Tuple, Set
import numpy as np
from pathlib import Path
import calc_similarity as cs
from scipy import stats
import ray
import csv

## Ontology and Avatar loading

In [2]:
# need to initialize the ontology for hpo3 (pyhpo) library (loads packaged HPO and
# disease assests into memory); follows a singleton pattern
_ = Ontology()

mecfs_pheno_file = Path("../data/mecfs-phenotypes.tsv")
mecfs_phenos = cs.load_phenos_by_group(str(mecfs_pheno_file))
all_pheno_profiles = cs.get_mecfs_pheno_combos(mecfs_phenos)
pene_hposet = HPOSet.from_queries(mecfs_phenos["PENE"])
mecfs_hposet = HPOSet.from_queries([p for subcat in mecfs_phenos.values() for p in subcat])

# add PENE phenotypes to all MECFS avatars as that is a manditory feature
for pheno_profile in all_pheno_profiles:
    pheno_profile.extend(mecfs_phenos["PENE"])

mecfs_hposet_avatars = [HPOSet.from_queries(p) for p in all_pheno_profiles]

## Disease Enrichment Analysis

HPO3 has a built-in disease enrichment calculation but it doesn't appear to support a process for correcting for multiple hypothesis testing (MHT),
at least as of the writing of this analysis. Implementing an erichment analysis (a.k.a. over representation analysis) to include correction for
MHT can be done in a straight forward manner. Having the p-values for all of the disease enrichment tests allows us to apply the
Benjamini and Hochberg (BH) correction process for controlling the false discovery rate. However, the HPO3 disease enrichment function only returns
the enriched diseases (p-value < 0.05) and the BH correction needs the p-values from all tests, not just the significant results. Thus, we have to
implement the disease enrichment procedure ourselves and then perform the correction.

In [3]:
# build background set of phenotypes (count), only focusing on non-modifier terms
# (i.e., terms that are children of HP:0000118 | Phenotypic abnormality)
total_phenos_count = 0
pheno_abnormality_term = Ontology.get_hpo_object("HP:0000118")
for term in Ontology:
    if pheno_abnormality_term in term.all_parents:
        total_phenos_count += 1

# create disease list where only non-modifier terms are used for describing the disease,
# for same reason we restrict the "Phenotypic abnormality" subsection of the ontology above
omim_id_only_nonmod_phenos = [(disease.name, set(disease.hpo_set().remove_modifier().serialize().split("+"))) for disease in Ontology.omim_diseases]

Setup a single function to compute the disease enrichment and BH correction for an MECFS avatar and use the python Ray library to facilitate parallelization (we'll need to perform this process for 110,160 avatars)

In [4]:
@ray.remote
def get_hypergeom_enrich(
    query_id: str,
    query: Set[str],
    diseases: List[Tuple[str, Set[str]]],
    total_phenos_count: int,
    alpha: float = 0.05,
) -> Tuple[str, List[Tuple[str, float, float]]]:
    # setup one time computation values for contingency table
    N = len(query)
    M = total_phenos_count

    # for each disease in the ontology build a contingency table
    # and compute one-sided Fisher's exact test (again, only using non-modifier phenotypes of the disease)
    raw_pvals = []
    for disease in diseases:
        # contingency table format
        # _____________________|_Query Phenotypes_|_Nonquery Phenotypes_|
        # Disease Phenotypes   |        x         |        n - x        |
        # Nondisease Phenotypes|      N - x       |    M - (n+N) + x    |
        #
        # x = count of phenotypes shared between the query phenotype set and the disease phenotype set
        # N = count of phenotypes in the query phenotype set
        # n = count of phenotypes in the disease phenotype set
        # M = count of all possible phenotypes that a disease or query phenotype set could have
        x = len(disease[1].intersection(query))
        n = len(disease[1])
        table = np.array([[x, n - x], [N - x, M - (n + N) + x]])
        raw_pvals.append(stats.fisher_exact(table, alternative="greater").pvalue)

    # apply Benjamini-Hochberg Procedure to adjust for multiple hypothesis testing
    adj_pvals = stats.false_discovery_control(raw_pvals)
    # filter for significantly enriched diseases after corretion for MHT
    sig_enrich = [(disease[0], raw_pval, adj_pval) for disease, raw_pval, adj_pval in zip(diseases, raw_pvals, adj_pvals) if adj_pval < alpha]
    return (
        query_id,
        sig_enrich
    )


### Enrichment calculation comparison

Start Ray instance locally for running computation in parallel

In [5]:
ray.init()

2025-11-11 15:27:37,095	INFO worker.py:2013 -- Started a local Ray instance.
/Users/brandon/miniforge3/envs/mecfs-pheno/lib/python3.10/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


Python version:,3.10.19
Ray version:,2.50.1


We want to compare our enrichment calulations to the established process from the HPO3 to ensure we didn't make any mistakes in the procedure

In [6]:
# test for enrichment using the PENE MECFS phenotypes list and the complete set of MECFS phenotypes using HPO3 stats package
model = hpostats.EnrichmentModel("omim")
pene_pyhpo_enrich = model.enrichment("hypergeom", pene_hposet)
mecfs_pyhpo_enrich = model.enrichment("hypergeom", mecfs_hposet)
print(f"Core PENE enriched diseases: {len(pene_pyhpo_enrich)}, MECFS all phenos enriched diseases: {len(mecfs_pyhpo_enrich)}")

# test the same using our reimplimentation
task_handles = [
    get_hypergeom_enrich.remote("PENE", set(pene_hposet.remove_modifier().serialize().split("+")), omim_id_only_nonmod_phenos, total_phenos_count, alpha=1.01),
    get_hypergeom_enrich.remote("MECFS", set(mecfs_hposet.remove_modifier().serialize().split("+")), omim_id_only_nonmod_phenos, total_phenos_count, alpha=1.01)
]
task_results = ray.get(task_handles)
pene_custom_enrich_cnt = sum(1 for testres in task_results[0][1] if testres[1] < 0.05)
mecfs_custom_enrich_cnt = sum(1 for testres in task_results[1][1] if testres[1] < 0.05)
print(f"Core PENE enriched diseases: {pene_custom_enrich_cnt}, MECFS all phenos enriched diseases: {mecfs_custom_enrich_cnt}")

Core PENE enriched diseases: 56, MECFS all phenos enriched diseases: 2413
Core PENE enriched diseases: 56, MECFS all phenos enriched diseases: 731


We get the same enrichment results when considering just the smaller list of the core MECFS PENE phenotypes but different when considering the larger MECFS phenotype set. Closer inspection of HPO3 shows that the disease enrichement test doesn't limit to the subset of phenotypes under `Phenotypic abnormality` like the custome function. 

### Avatar Disease Enrichment with BH correction 

In [12]:
# schedule avatar disease enrichment computation with running Ray instance
# note: this schedules 110,160 enrichment tasks to be run and Ray uses all available
# resources to complete tasks, you have been notified
dis_enrich_tasks = []
for idx, avatar in enumerate(mecfs_hposet_avatars):
    dis_enrich_tasks.append(get_hypergeom_enrich.remote(f"Avatar {idx}", set(avatar.remove_modifier().serialize().split("+")), omim_id_only_nonmod_phenos, total_phenos_count))

# ask Ray to get results from all enrichment tasks,
# returned in a list with the same order as the scheduled tasks
# this will take ~4.5 hours on a 16-core 32GB RAM machine 
enriched_dis_per_avatar = ray.get(dis_enrich_tasks)

# write findings to file for backup and later use
enrich_res_file = Path("../data/results/mecfs-avatars-disease-enrichment.tsv")
with enrich_res_file.open("wt") as outfile:
    outfile.write("Avatar ID\tDisease\tp-value\tadj. p-value\n")
    for enrich_res in enriched_dis_per_avatar:
        avatar_id = enrich_res[0]
        for enrich_dis_res in enrich_res[1]:
            formatted_output = [avatar_id, enrich_dis_res[0], f"{enrich_dis_res[1]:.14f}", f"{enrich_dis_res[2]:.14f}"]
            outfile.write("\t".join(formatted_output) + "\n")


(raylet) Spilled 2530 MiB, 1885 objects, write throughput 284 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 4099 MiB, 3053 objects, write throughput 257 MiB/s.
(raylet) Spilled 8195 MiB, 6103 objects, write throughput 241 MiB/s.
(raylet) Spilled 16385 MiB, 12201 objects, write throughput 240 MiB/s.
(raylet) Spilled 32774 MiB, 24403 objects, write throughput 257 MiB/s.
(raylet) Spilled 65542 MiB, 48801 objects, write throughput 269 MiB/s.
(raylet) Spilled 131073 MiB, 97592 objects, write throughput 244 MiB/s.


After all the compute is done we can shutdown the Ray instance

## Disease Ranking

Since the number of avatars in which a disease is enriched and the enrichment score is variable we need a ranking scheme that will account for these factors when producing an overall rank of enriched diseases. For this process we leverage a weighted version of the [mean reciprical rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank). The weight is derived from the number of avatars which a given disease was phenotypicially significantly enriched for. Some diseases are significantly enriched for a subset of avatars, but enrich highly for that subset. The weighting helps rank those diseases higher in the overall ranking over diseases that enrich for more avatars but not as highly.

**NOTE:** see data/weighted-ranking-demo.xlsx for a walkthrough of the calculation to weight the median reciprical ranks for ranking diseases best matching MECFS phenotypic avatars.

In [ ]:
import pandas as pd
from pathlib import Path
# format data into pandas dataframe
enrich_res_file = Path("../data/results/mecfs-avatars-disease-enrichment.tsv")
enrich_df = pd.read_table(enrich_res_file, index_col=None)

Compute the reciprical of the rank for each disease for each avatar and store with other information

In [37]:

# group by avatar and store reciprical of the rank for each disease within that avatar
enrich_df["in_avatar_recip_rank"] = 1.0/enrich_df.groupby("Avatar ID")["adj. p-value"].rank("first")

# compute the mean reciprical rank per disease
res_df = enrich_df.groupby("Disease")[["in_avatar_recip_rank"]].mean()
res_df.rename(columns={"in_avatar_recip_rank": "mrr"}, inplace=True)

# Calculate the weight (the percentage impact to the weighted average), the
# weighting strategy is based on this answer https://math.stackexchange.com/a/1577209 to a similar problem.
# This computes weights for helping with ranking by accounting for difference in number of avatars
# each disease is enriched in.
avatar_cnt_df = enrich_df.groupby("Disease").count().rename(columns={"Avatar ID": "avatar_cnt"}).drop(columns=["p-value", "adj. p-value", "in_avatar_recip_rank"])
avatar_cnt_df["percent_enriched"] = avatar_cnt_df["avatar_cnt"]/110160.0
mean_percent_positive = avatar_cnt_df["percent_enriched"].mean()
avatar_cnt_df["raw_weight"] = (
    (avatar_cnt_df["percent_enriched"] - mean_percent_positive)
    * (avatar_cnt_df["avatar_cnt"] / (len(avatar_cnt_df) * 110160.0))
    / mean_percent_positive
)

# the raw weights will be centered around zero, we need to rescale them between zero and one
# to appropriately apply the weights to the MRR
avatar_cnt_df['scaled_weight'] = (avatar_cnt_df['raw_weight'] - avatar_cnt_df['raw_weight'].min()) / (avatar_cnt_df['raw_weight'].max() - avatar_cnt_df['raw_weight'].min())

# bring everything together and compute the weighted MRR
res_merged_df = pd.merge(res_df, avatar_cnt_df, how="inner", on="Disease")
res_merged_df = res_merged_df.reset_index()
res_merged_df["weighted_mrr"] = res_merged_df["mrr"] * res_merged_df["scaled_weight"]

add some support functions for easy formatting

In [6]:
# add OMIM object to primary findings df for efficient lookup
def omim_lookup_by_name(disease):
    for omim_dis in Ontology.omim_diseases:
        if omim_dis.name == disease:
            return omim_dis
        

def omim_lookup_by_id(omim_id):
    return Omim.get(omim_id)


# highlighting function for rows in output
def highlight_row(row):
    enrich_dis = omim_lookup_by_name(row["Disease"])
    if enrich_dis in findings_disease_set:
        return ["background-color: lightblue"] * len(row)

    return [""] * len(row)

Add highlighting of diseases found in participants in cohort in the list of enriched diseases found by the avatar analysis

In [ ]:
# pretty up dataframe for final output
col_names = {
    "weighted_mrr": "Weighted Mean Reciprical Rank",
    "avatar_cnt": "# Avatars Enriched In",
    "mrr": "Mean Reciprical Rank",
    "scaled_weight": "Weight"
}
output_df = res_merged_df[["Disease", *col_names.keys()]].copy()
output_df.rename(columns=col_names, inplace=True)
output_df.sort_values("Weighted Mean Reciprical Rank", inplace=True, ascending=False)
output_df["Rank"] = output_df["Weighted Mean Reciprical Rank"].rank(ascending=False)

# load disease info from table 1
primary_disease_info = pd.read_excel(
    "../data/mecfs-samples/primary-disease-info.xlsx"
)

findings_disease_set = set(primary_disease_info["OMIM"].apply(omim_lookup_by_id).to_list())

# apply styling per row
styled_df = output_df.style.apply(highlight_row, axis=1)

# write to excel file for supp.
styled_df.to_excel(
    "../data/results/Supp. Table - MECFS avatars and ranked disease enrichment table.xlsx",
    index=False,
    columns=[
        "Rank",
        "Disease",
        "Weighted Mean Reciprical Rank", 
        "# Avatars Enriched In", 
        "Mean Reciprical Rank", 
        "Weight"
    ]
)


# Disease Enrichment analysis of Participants

In the same fashion as the disease enrichment analysis of the ME/CFS avatars we can perform the calculations based on the phenotypes of the cohort's participants.

## Participant Disease Enrichment Analysis

In [7]:
# helper function for reading participant phenotypes from file
def parse_n_clean_phenos(participant_phenos: str) -> HPOSet:
    return HPOSet.from_queries(participant_phenos.replace(" ", "").split(","))


data_path = "../data/mecfs-samples/patient_hpo_summaries.csv"
participant_phenos = []
with open(data_path, "r", newline="") as fp:
    reader = csv.DictReader(fp)
    for row in reader:
        participant_phenos.append(
            (row["Paper_ID"], parse_n_clean_phenos(row["HPO_Terms"]))
        )

just like we did with the avatars we can ship the enrichment process off to Ray and let it do it quickly in parallel.

In [8]:
# Lazy developer copy and variable renaming from above (I know a generic function could be used here but the analysis doesn't warrant the time to make it so)
# schedule participant disease enrichment computation with running Ray instance
# note: this schedules 31 enrichment tasks to be run and Ray uses all available
# resources to complete tasks, you have been notified
dis_enrich_tasks = []
for idx, part_phenos in participant_phenos:
    dis_enrich_tasks.append(get_hypergeom_enrich.remote(f"{idx}", set(part_phenos.remove_modifier().serialize().split("+")), omim_id_only_nonmod_phenos, total_phenos_count))

# ask Ray to get results from all enrichment tasks,
# returned in a list with the same order as the scheduled tasks
# this will only take a few seconds on a 16-core 32GB RAM machine
enriched_dis_per_participant = ray.get(dis_enrich_tasks)

# write findings to file for backup and later use
enrich_res_file = Path("../data/results/participants-disease-enrichment.tsv")
with enrich_res_file.open("wt") as outfile:
    outfile.write("Participant ID\tDisease\tp-value\tadj. p-value\n")
    for enrich_res in enriched_dis_per_participant:
        p_id = enrich_res[0]
        for enrich_dis_res in enrich_res[1]:
            formatted_output = [p_id, enrich_dis_res[0], f"{enrich_dis_res[1]:.14f}", f"{enrich_dis_res[2]:.14f}"]
            outfile.write("\t".join(formatted_output) + "\n")

## Participant Disease Ranking

In [9]:
import pandas as pd

# group by participant and store reciprical of the rank for each disease within that participant
enrich_df = pd.read_table(enrich_res_file)
enrich_df["in_participant_recip_rank"] = 1.0/enrich_df.groupby("Participant ID")["adj. p-value"].rank("first")

# compute the mean reciprical rank per disease
res_df = enrich_df.groupby("Disease")[["in_participant_recip_rank"]].mean()
res_df.rename(columns={"in_participant_recip_rank": "mrr"}, inplace=True)

# Calculate the weight (the percentage impact to the weighted average), the
# weighting strategy is based on this answer https://math.stackexchange.com/a/1577209 to a similar problem.
# This computes weights for helping with ranking by accounting for difference in number of participants
# each disease is enriched in.
participant_cnt_df = enrich_df.groupby("Disease").count().rename(columns={"Participant ID": "participant_cnt"}).drop(columns=["p-value", "adj. p-value", "in_participant_recip_rank"])
participant_cnt_df["percent_enriched"] = participant_cnt_df["participant_cnt"]/110160.0
mean_percent_positive = participant_cnt_df["percent_enriched"].mean()
participant_cnt_df["raw_weight"] = (
    (participant_cnt_df["percent_enriched"] - mean_percent_positive)
    * (participant_cnt_df["participant_cnt"] / (len(participant_cnt_df) * 110160.0))
    / mean_percent_positive
)

# the raw weights will be centered around zero, we need to rescale them between zero and one
# to appropriately apply the weights to the MRR
participant_cnt_df['scaled_weight'] = (participant_cnt_df['raw_weight'] - participant_cnt_df['raw_weight'].min()) / (participant_cnt_df['raw_weight'].max() - participant_cnt_df['raw_weight'].min())

# bring everything together and compute the weighted MRR
res_merged_df = pd.merge(res_df, participant_cnt_df, how="inner", on="Disease")
res_merged_df = res_merged_df.reset_index()
res_merged_df["weighted_mrr"] = res_merged_df["mrr"] * res_merged_df["scaled_weight"]

# pretty up dataframe for final output
col_names = {
    "weighted_mrr": "Weighted Mean Reciprical Rank",
    "participant_cnt": "# Participants Enriched In",
    "mrr": "Mean Reciprical Rank",
    "scaled_weight": "Weight"
}
output_df = res_merged_df[["Disease", *col_names.keys()]].copy()
output_df.rename(columns=col_names, inplace=True)
output_df.sort_values("Weighted Mean Reciprical Rank", inplace=True, ascending=False)
output_df["Rank"] = output_df["Weighted Mean Reciprical Rank"].rank(ascending=False)

# load disease info from table 1
primary_disease_info = pd.read_excel(
    "../data/mecfs-samples/primary-disease-info.xlsx"
)

# add OMIM object to primary findings df for efficient lookup
def omim_lookup_by_name(disease):
    for omim_dis in Ontology.omim_diseases:
        if omim_dis.name == disease:
            return omim_dis
        

def omim_lookup_by_id(omim_id):
    return Omim.get(omim_id)


findings_disease_set = set(primary_disease_info["OMIM"].apply(omim_lookup_by_id).to_list())

# highlighting function for rows in output
def highlight_row(row):
    enrich_dis = omim_lookup_by_name(row["Disease"])
    if enrich_dis in findings_disease_set:
        return ["background-color: lightblue"] * len(row)

    return [""] * len(row)

# apply styling per row
styled_df = output_df.style.apply(highlight_row, axis=1)

# write to excel file for supp.
styled_df.to_excel(
    "../data/results/Supp. Table - MECFS participants ranked disease enrichment table.xlsx",
    index=False,
    columns=[
        "Rank",
        "Disease",
        "Weighted Mean Reciprical Rank", 
        "# Participants Enriched In", 
        "Mean Reciprical Rank", 
        "Weight"
    ]
)

In [10]:
ray.shutdown()

I0000 00:00:1762896573.102194 10089496 chttp2_transport.cc:1182] ipv4:127.0.0.1:60878: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {file:"external/com_github_grpc_grpc/src/core/ext/transport/chttp2/transport/chttp2_transport.cc", file_line:1171, created_time:"2025-11-11T15:29:33.102182-06:00", http2_error:2, grpc_status:14}
